# Setup

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris()

iris_frame = pd.DataFrame(iris.data, columns=iris.feature_names)

iris_frame['target'] = iris.target

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        self.load_dataset()
        self.train()

In [4]:
class PipelineWithConfig(SimplePipeline):
    def __init__(self, config):
        super().__init__()
        self.config = config
            
    def train(self, algorithm=LogisticRegression):
        self.model = algorithm(solver=self.config.get('solver'),
                               multi_class=self.config.get('multi_class'))
        self.model.fit(self.X_train, self.y_train)

# Test

In [5]:
import unittest

In [6]:
ENABLED_MODEL_SOLVERS = {'lbfgs', 'newton-cg'}


class TestIrisConfig(unittest.TestCase):
    def setUp(self):
        # We prepare the pipeline for use in the tests
        config = {'solver': 'lbfgs', 'multi_class': 'auto'}
        self.pipeline = PipelineWithConfig(config=config)
        self.pipeline.run_pipeline()
    
    def test_pipeline_config(self):
        model_params = self.pipeline.model.get_params()
    
        self.assertTrue(model_params['solver'] in ENABLED_MODEL_SOLVERS)

In [7]:
import sys

suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisConfig)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.020s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>